### Using Redis with Feast
In this section we will see how Redis is being used with [Feast](https://feast.dev/). Since Feast 0.11, Redis is a first class citizen in Feast's current architecture, and requires no addtional coding in order to use it.

<img src="./images/feast_data_model.png" width="800">

In [1]:
%cd feature_repo_cloud

/home/dvirdukhan/Code/mlops-world-demo/feature_repo_cloud


In [2]:
!pygmentize feature_store.yaml

project: demo_repo
registry: data/registry.db
provider: local
online_store:
    type: redis
    connection_string: ${REDIS_HOST}:${REDIS_PORT},password=${REDIS_PASSWORD}


In [3]:
# !pygmentize example.py

In [2]:
!feast apply

Created entity driver
Created feature view driver_hourly_stats
Created feature service driver_activity

Deploying infrastructure for driver_hourly_stats


In [3]:

!feast materialize-incremental $(date -u +"%Y-%m-%dT%H:%M:%S")

Materializing 1 feature views to 2022-06-15 00:31:45+03:00 into the redis online store.

driver_hourly_stats from 2022-06-07 21:31:46+03:00 to 2022-06-15 00:31:45+03:00:
100%|█████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.27it/s]


Create a redis client to check the network latency

In [10]:
import redis
import os
r = redis.Redis(host = os.environ['REDIS_HOST'], port = os.environ['REDIS_PORT'], password=os.environ['REDIS_PASSWORD'] )

In [12]:
%%time
r.ping()

CPU times: user 3.18 ms, sys: 0 ns, total: 3.18 ms
Wall time: 161 ms


True

In [5]:
from pprint import pprint
from feast import FeatureStore

store = FeatureStore(repo_path=".")

In [9]:
%%time
feature_vector = store.get_online_features(
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
    ],
    entity_rows=[
        {"driver_id": 0},
        {"driver_id": 1002},
        {"driver_id": 1003},
        {"driver_id": 1004},
        {"driver_id": 1005},
    ],
)

CPU times: user 1.7 ms, sys: 3.65 ms, total: 5.35 ms
Wall time: 161 ms


In [7]:
pprint(feature_vector.to_dict())

{'acc_rate': [None,
              0.1812489926815033,
              0.2525552213191986,
              0.2085496187210083,
              0.6677494645118713],
 'avg_daily_trips': [None, 787, 596, 832, 606],
 'conv_rate': [None,
               0.3829907476902008,
               0.4977691173553467,
               0.40254834294319153,
               0.07789330184459686],
 'driver_id': [0, 1002, 1003, 1004, 1005]}


### Summary
We saw the usage of Feast in order to get feature vectors into redis. 

Target persona: MLOps engineers/Data engineers/Platform architects.
Consider using this approach if:
1. You are looking for fast development/deployment time - not intending to write your own feature store layer/product.
    1. You are not in control of your data path.
    1. You are "OK" with 3rd party data modeling/access patterns/data serilization or compression.